## Group comparison in python 

Import libaries

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scikit_posthocs as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.diagnostic import het_breuschpagan

Generate Example Data

In [ ]:
np.random.seed(42)
data = pd.DataFrame({
    'Group': np.random.choice(['A', 'B'], 100),
    'Score': np.random.normal(50, 10, 100),
    'Subject': np.arange(100),
    'Time1': np.random.normal(50, 10, 100),
    'Time2': np.random.normal(52, 10, 100),
    'Covariate1': np.random.normal(100, 15, 100),
    'Covariate2': np.random.normal(100, 15, 100),})

# longitudinal dataframe
data_long = data.melt(id_vars=['Group', 'Subject', 'Score', 'Covariate1', 'Covariate2'], value_vars=['Time1', 'Time2'], 
                      var_name='Time', value_name='Time_Score')

### Assumption Testing

Normality Tests

In [ ]:
# Shapiro-Wilk test
for group in data['Group'].unique():
    group_data = data[data['Group'] == group]['Score']
    stat, p = stats.shapiro(group_data)
    print(f"Shapiro-Wilk test for Group {group}: stat={stat:.4f}, p={p:.4f}")

In [ ]:
# Q-Q plots
plt.figure(figsize=(15, 5))
for i, group in enumerate(data['Group'].unique()):
    plt.subplot(1, 3, i+1)
    group_data = data[data['Group'] == group]['Score']
    sm.qqplot(group_data, line='s', ax=plt.gca())
    plt.title(f'Q-Q Plot for Group {group}')

In [ ]:
# Histograms
plt.figure(figsize=(15, 5))
for i, group in enumerate(data['Group'].unique()):
    plt.subplot(1, 3, i+1)
    group_data = data[data['Group'] == group]['Score']
    sns.histplot(group_data, kde=True, ax=plt.gca())
    plt.title(f'Histogram for Group {group}')

Homogeneity of Variance

In [ ]:
# Levene's test
levene_stat, levene_p = stats.levene(*[data[data['Group'] == group]['Score'] for group in data['Group'].unique()])
print(f"Levene's test: stat={levene_stat:.4f}, p={levene_p:.4f}")

# Bartlett's test
bartlett_stat, bartlett_p = stats.bartlett(*[data[data['Group'] == group]['Score'] for group in data['Group'].unique()])
print(f"Bartlett's test: stat={bartlett_stat:.4f}, p={bartlett_p:.4f}")

Linearity (for ANCOVA and regression)

In [ ]:
sns.pairplot(data, x_vars=["Covariate1", "Covariate2"], y_vars=["Score"], hue="Group", height=5, aspect=.8, kind="reg")

Homogeneity of Regression Slopes (for ANCOVA)

In [ ]:
ancova_interaction = smf.ols('Score ~ Group * Covariate1 + Group * Covariate2', data=data).fit()
print(ancova_interaction.summary())

Independence of Covariates and Groups (for ANCOVA)

In [ ]:
for covar in ['Covariate1', 'Covariate2']:
    anova_result = pg.anova(data=data, dv=covar, between='Group')
    print(f"ANOVA testing if {covar} differs by Group:")
    print(anova_result)

Sphericity (for repeated measures ANOVA)

In [ ]:
print("With only time points, sphericity is automatically met.")
sphericity_test = pg.sphericity(data_long, dv='Time_Score', within='Time', subject='Subject')
print(sphericity_test)

 Normality of Residuals

In [ ]:
# Fit a model
model = smf.ols('Score ~ Group + Covariate1 + Covariate2', data=data).fit()
residuals = model.resid

# Shapiro-Wilk test on residuals
shapiro_stat, shapiro_p = stats.shapiro(residuals)
print(f"Shapiro-Wilk test on residuals: stat={shapiro_stat:.4f}, p={shapiro_p:.4f}")

# Q-Q plot of residuals
plt.figure(figsize=(5, 3))
sm.qqplot(residuals, line='s', ax=plt.gca())
plt.title('Q-Q Plot of Residuals')

# Histogram of residuals
plt.figure(figsize=(5, 3))
sns.histplot(residuals, kde=True)
plt.title('Histogram of Residuals')

# Homoscedasticity
bp_test = het_breuschpagan(residuals, model.model.exog)
print(f"Breusch-Pagan test: LM stat={bp_test[0]:.4f}, p={bp_test[1]:.4f}")

### Group comparison tests

Independent t-test

In [ ]:
ind_ttest = stats.ttest_ind(
    data.loc[data['Group'] == 'A', 'Score'],
    data.loc[data['Group'] == 'B', 'Score'],
    equal_var=True)
print("Independent t-test:", ind_ttest)

Paired t-test

In [ ]:
paired_ttest = stats.ttest_rel(data['Time1'], data['Time2'])
print("Paired t-test:", paired_ttest)

One-way ANOVA

In [ ]:
# statsmodels
anova = smf.ols('Score ~ Group', data=data).fit()
anova_result = sm.stats.anova_lm(anova, typ=2)

# pingouin
anova_result = pg.anova(data=data, dv='Score', between='Group').round(3)

print("One-way ANOVA:", anova_result)

Repeated Measures ANOVA

In [ ]:
rm_anova = pg.rm_anova(data=data_long, dv='Time_Score', within='Time', subject='Subject', detailed=True)
print("Repeated Measures ANOVA:", rm_anova)

Mixed ANOVA

In [ ]:
mixed_anova = pg.mixed_anova(data=data_long, dv='Time_Score', within='Time', between='Group', subject='Subject')
print("Mixed ANOVA:", mixed_anova)

ANCOVA

In [ ]:
# statsmodels
ancova = smf.ols('Score ~ Group + Covariate1 + Covariate2', data=data).fit()
ancova_result = sm.stats.anova_lm(ancova, typ=2)

# pingouin
ancova_result = pg.ancova(data=data, dv='Score', between='Group', covar=['Covariate1', 'Covariate2']).round(3) 

print("ANCOVA:", ancova_result)

Linear Mixed Model (LMM)

In [ ]:
lmm = smf.mixedlm('Score ~ Group + Covariate1 + Covariate2', data, groups=data['Subject']).fit()
print("Linear Mixed Model:", lmm.summary())

# longitudinal data:
lmm_long = smf.mixedlm('Time_Score ~ Time * Group + Covariate1 + Covariate2', data_long, groups=data_long['Subject']).fit()
print("Linear Mixed Model:", lmm_long.summary())

Generalized Linear Mixed Model (GLMM)

In [ ]:
glmm = smf.glm('Score ~ Group + Covariate1 + Covariate2', data, groups=data['Subject'], family=sm.families.Binomial()).fit()
print("Generalized Linear Mixed Model:", glmm.summary())

# longitudinal data:
glmm_long = smf.glm('Time_Score ~ Time * Group + Covariate1 + Covariate2', data_long, groups=data_long['Subject'], family=sm.families.Binomial()).fit()
print("Generalized Linear Mixed Model:", glmm_long.summary())

### Post-hoc tests

Post-hoc for ANOVA

In [ ]:
# Fisher's LSD
lsd = pg.pairwise_tests(data=data, dv='Score', between='Group', parametric=True)
print("Fisher's LSD:", lsd)

# Turkey
tukey = sp.posthoc_tukey_hsd(data, val_col='Score', group_col='Group')
print("Turkey HSD:", tukey)

tukey = pg.pairwise_tukey(data, dv='Score', between='Group')
print("Turkey HSD:", tukey)

# Holm correction
holm = sp.posthoc_ttest(data, val_col='Score', group_col='Group', p_adjust='holm')
print("Holm correction:", holm)

holm = pg.pairwise_ttests(data=data, dv='Score', between='Group', padjust='holm')
print("Holm correction:",holm)

# Dunnett correction
dunnet = stats.dunnett(data.loc[data['Group'] == 'A', 'Score'], control=data.loc[data['Group'] == 'B', 'Score'])
print("Dunnet correction:", dunnet)

# Bonferonni correction
bonf = pg.pairwise_ttests(data=data, dv='Score', between='Group', padjust='bonf')
print("Bonferroni correction:", bonf)

non- parametric tests

In [ ]:
# Kruskal-Wallis
kruskal = pg.kruskal(data=data, dv='Score', between='Group')
print("Kruskal-Wallis:", kruskal)

# Mann-Whitney U test
mwu = pg.mwu(data.loc[data['Group'] == 'A', 'Score'], data.loc[data['Group'] == 'B', 'Score'])
print("Mann-Whitney U test:", mwu)

Post-hoc for repeated measure ANOVA

In [ ]:
posthoc_rm = pg.pairwise_ttests(data=data_long, dv='Time_Score', within='Time', subject='Subject', padjust='bonf')
print(posthoc_rm)

Post-hoc for mixed ANOVA

In [ ]:
posthoc_mixed = pg.pairwise_ttests(data=data_long, dv='Time_Score', between='Group', within='Time', subject='Subject', padjust='bonf')
print(posthoc_mixed)

Post-hoc with covariates: no easy solution in python

maybe check: https://eshinjolly.com/pymer4/index.html 